In [1]:
// Load the EnvConfigHelper class from our project
#load "../EnvConfigHelper.cs"

  // Load configuration from .env file
  var modelDeploymentName = EnvConfigHelper.AzureOpenAIDeploymentName;
  var azureOpenAIEndpoint = EnvConfigHelper.AzureOpenAIEndpoint;
  var azureOpenAIKey = EnvConfigHelper.AzureOpenAIKey;


// Display loaded configuration (without sensitive data)
Console.WriteLine("🔑 Configuration loaded successfully!");
Console.WriteLine($"Azure OpenAI Endpoint: {azureOpenAIEndpoint}");
Console.WriteLine($"Deployment Name: {modelDeploymentName}");
Console.WriteLine($"API Key: {(string.IsNullOrEmpty(azureOpenAIKey) ? "❌ Not loaded" : "✅ Loaded (hidden)")}");

✅ Loaded .env from: d:\VSC\semantic-kernel-in-action-1-fundamentals-3836112\src\.env
🔑 Configuration loaded successfully!
Azure OpenAI Endpoint: https://dis-openai-0705.openai.azure.com/
Deployment Name: gpt-4.1
API Key: ✅ Loaded (hidden)


In [3]:
#r "nuget: Microsoft.SemanticKernel, *"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.61.0-preview"


using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins.Core;
using System.ComponentModel;
using System.IO;


Installed Packages Microsoft.SemanticKernel, 1.61.0 Microsoft.SemanticKernel.Plugins.Core, 1.61.0-preview

In [4]:
 // Create the builder
var builder = Kernel.CreateBuilder();

// Build the kernel
var kernel = builder.Build();



In [5]:
var textPlugin = new TextPlugin();
kernel.Plugins.AddFromObject(textPlugin, "text");

var textResult = await kernel.InvokeAsync("text", "uppercase", new() {
    ["input"] = "polyglot rocks"
});
Console.WriteLine(textResult.GetValue<string>());



POLYGLOT ROCKS


In [9]:
using Microsoft.SemanticKernel;
using System.ComponentModel;

public class MyMathPlugin
{
    [KernelFunction, Description("Adds two integers")]
    public int Add(int a, int b) => a + b;

    [KernelFunction, Description("Subtracts second integer from the first")]
    public int Subtract(int a, int b) => a - b;

    [KernelFunction, Description("Multiplies two integers")]
    public int Multiply(int a, int b) => a * b;

    [KernelFunction, Description("Divides first number by second")]
    public double Divide(double a, double b) => b != 0 ? a / b : double.NaN;
}

In [11]:
 // Create the builder
var builder = Kernel.CreateBuilder();

// Build the kernel
var kernel = builder.Build();

var mathPlugin = new MyMathPlugin();

kernel.Plugins.AddFromObject(mathPlugin, "math");

var addResult = await kernel.InvokeAsync("math", "Add", new() {
    ["a"] = 12,
    ["b"] = 8
});
Console.WriteLine($"➕ Add: {addResult.GetValue<int>()}");

var divResult = await kernel.InvokeAsync("math", "Divide", new() {
    ["a"] = 20,
    ["b"] = 4
});
Console.WriteLine($"➗ Divide: {divResult.GetValue<double>()}");



➕ Add: 20
➗ Divide: 5


In [12]:
 // Create the builder
var builder = Kernel.CreateBuilder();

// Build the kernel
var kernel = builder.Build();

var timePlugin = new TimePlugin();
kernel.Plugins.AddFromObject(timePlugin, "time");

var timeResult = await kernel.InvokeAsync("time", "date");
Console.WriteLine($"📅 Today is: {timeResult.GetValue<string>()}");


📅 Today is: Thursday, 24 July 2025


In [13]:
 // Create the builder
var builder = Kernel.CreateBuilder();

// Build the kernel
var kernel = builder.Build();

// Check if the plugin with the name "file" already exists
if (!kernel.Plugins.Any(p => p.Name == "file"))
{
    var filePlugin = new FileIOPlugin();
    kernel.Plugins.AddFromObject(filePlugin, "file");
}

var writeResult = await kernel.InvokeAsync("file", "write", new() {
    ["path"] = "./test.txt",
    ["content"] = "Hello from Semantic Kernel!"
});
Console.WriteLine("✅ File written.");


✅ File written.


In [14]:
 // Create the builder
var builder = Kernel.CreateBuilder();

// Build the kernel
var kernel = builder.Build();

// ✅ Register HttpPlugin only if not already added
if (!kernel.Plugins.Any(p => p.Name == "http"))
{
    var httpPlugin = new HttpPlugin();
    kernel.Plugins.AddFromObject(httpPlugin, "http");
}

// ✅ Invoke GET with headers (GitHub requires User-Agent)
var httpResult = await kernel.InvokeAsync("http", "get", new()
{
    ["uri"] = "https://api.github.com",
    ["headers"] = "{\"User-Agent\": \"SemanticKernel-Agent\"}"
});

Console.WriteLine($"🌐 Response:\n{httpResult.GetValue<string>()}");


🌐 Response:
{"current_user_url":"https://api.github.com/user","current_user_authorizations_html_url":"https://github.com/settings/connections/applications{/client_id}","authorizations_url":"https://api.github.com/authorizations","code_search_url":"https://api.github.com/search/code?q={query}{&page,per_page,sort,order}","commit_search_url":"https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}","emails_url":"https://api.github.com/user/emails","emojis_url":"https://api.github.com/emojis","events_url":"https://api.github.com/events","feeds_url":"https://api.github.com/feeds","followers_url":"https://api.github.com/user/followers","following_url":"https://api.github.com/user/following{/target}","gists_url":"https://api.github.com/gists{/gist_id}","hub_url":"https://api.github.com/hub","issue_search_url":"https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}","issues_url":"https://api.github.com/issues","keys_url":"https://api.github.com/user/keys",

In [29]:
using Microsoft.SemanticKernel;
using System.ComponentModel;


 // Create the builder
var builder = Kernel.CreateBuilder();

// Build the kernel
var kernel = builder.Build();

// Define a simple EmailPlugin with a generateEmail function
public class EmailPlugin
{
    [KernelFunction, Description("Generates an email message for the recipient")]
    public string GenerateEmail(string recipient, string message)
    {
        return $"To: {recipient}\n\n{message}\n\nBest regards,\nYour SK Agent";
    }
}

if (!kernel.Plugins.Any(p => p.Name == "email"))
{
    var emailPlugin = new EmailPlugin();
    kernel.Plugins.AddFromObject(emailPlugin, "email");
}

var emailResult = await kernel.InvokeAsync("email", "GenerateEmail", new() {
    ["recipient"] = "kenneth@example.com",
    ["message"] = "Your SK agent is ready!"
});
Console.WriteLine(emailResult.GetValue<string>());



To: kenneth@example.com

Your SK agent is ready!

Best regards,
Your SK Agent


In [34]:
//#r "nuget: Microsoft.SemanticKernel.PromptTemplates.Handlebars, *"

using Microsoft.SemanticKernel.PromptTemplates.Handlebars;

var userData = new Dictionary<string, object>
{
    { "user.name", "John Doe" },
    { "user.role", "Developer" }
};

 // Create the builder
var builder = Kernel.CreateBuilder();

// Build the kernel
var kernel = builder.Build();

var templateFactory = new HandlebarsPromptTemplateFactory();
var promptTemplateConfig = new PromptTemplateConfig {
    Template = "{{user.name}} is {{user.role}}",
    TemplateFormat = "handlebars",
    Name = "UserIntro"
};
var promptTemplate = templateFactory.Create(promptTemplateConfig);
var kernelArgs = new KernelArguments(userData);
var renderedTemplate = await promptTemplate.RenderAsync(kernel, kernelArgs);

Console.WriteLine(renderedTemplate);



is
